In [ ]:
import os
os.environ['OPENAI_API_KEY'] = "sk-proj-"

In [ ]:
!pip install langchain-openai langchain-core langchain-community

In [ ]:
import requests
import json
from requests import get
from langchain_core.tools import tool , InjectedToolArg
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage ,ToolMessage
from typing import Annotated


In [ ]:
# First Creating a tool that get the weather in celcius

@tool
def get_weather(location: str) -> dict :
  """This tool get the current weather temperature in celcius"""
  API_KEY = 'f009308'
  url = f"http://api.weatherapi.com/v1/current.json?key={API_KEY}&q={location}"
  response = requests.get(url)

  return response.json()

@tool
def conver_c_to_f (temperature : Annotated[float , InjectedToolArg]) -> float :
  """This tool convert celcius to fahrenheit"""
  return round((temperature * 9/5) + 32,2)



In [ ]:
result = conver_c_to_f.invoke({'temperature' : 23.4})

In [ ]:
print(conver_c_to_f.args)

{'temperature': {'title': 'Temperature', 'type': 'number'}}


In [ ]:
weather = get_weather.invoke({'location':'London'})

In [ ]:
weather

{'location': {'name': 'London',
  'region': 'City of London, Greater London',
  'country': 'United Kingdom',
  'lat': 51.5171,
  'lon': -0.1062,
  'tz_id': 'Europe/London',
  'localtime_epoch': 1755068901,
  'localtime': '2025-08-13 08:08'},
 'current': {'last_updated_epoch': 1755068400,
  'last_updated': '2025-08-13 08:00',
  'temp_c': 18.4,
  'temp_f': 65.1,
  'is_day': 1,
  'condition': {'text': 'Partly cloudy',
   'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png',
   'code': 1003},
  'wind_mph': 3.1,
  'wind_kph': 5.0,
  'wind_degree': 137,
  'wind_dir': 'SE',
  'pressure_mb': 1014.0,
  'pressure_in': 29.94,
  'precip_mm': 0.0,
  'precip_in': 0.0,
  'humidity': 88,
  'cloud': 50,
  'feelslike_c': 18.4,
  'feelslike_f': 65.1,
  'windchill_c': 24.0,
  'windchill_f': 75.2,
  'heatindex_c': 25.2,
  'heatindex_f': 77.3,
  'dewpoint_c': 12.8,
  'dewpoint_f': 55.0,
  'vis_km': 10.0,
  'vis_miles': 6.0,
  'uv': 0.7,
  'gust_mph': 3.7,
  'gust_kph': 5.9,
  'short_rad': 178.14,
  'dif

In [ ]:
llm = ChatOpenAI()

In [ ]:
llm_with_tool = llm.bind_tools([get_weather , conver_c_to_f])

In [ ]:
message = [HumanMessage("What is the weather in the pakistan in the celcius also convert this celcius into farenheit")]

In [ ]:
message

[HumanMessage(content='What is the weather in the pakistan in the celcius also convert this celcius into farenheit', additional_kwargs={}, response_metadata={})]

In [ ]:
ai_message = llm_with_tool.invoke(message)

In [ ]:
ai_message

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_7v9aAPkIJt0OTbdOEJMTl0U1', 'function': {'arguments': '{"location": "Pakistan"}', 'name': 'get_weather'}, 'type': 'function'}, {'id': 'call_Sne8pxvCBF1CANvFAJuPxi61', 'function': {'arguments': '{}', 'name': 'conver_c_to_f'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 93, 'total_tokens': 137, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C3zwKSPXtHSN2vFw54B3OvOZ70voI', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--dce5517a-aa76-4d60-ba2d-08b9fd85ddd3-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'Pakistan'}, 'id': 'call_7v9aAPkIJt0OTbdOEJMTl0U1', 'type': 'to

In [ ]:
ai_message.tool_calls

[{'name': 'get_weather',
  'args': {'location': 'Pakistan'},
  'id': 'call_7v9aAPkIJt0OTbdOEJMTl0U1',
  'type': 'tool_call'},
 {'name': 'conver_c_to_f',
  'args': {},
  'id': 'call_Sne8pxvCBF1CANvFAJuPxi61',
  'type': 'tool_call'}]

In [ ]:
get_weather.invoke(ai_message.tool_calls[0])

ToolMessage(content='{"location": {"name": "Islamabad", "region": "Islamabad", "country": "Pakistan", "lat": 33.7, "lon": 73.167, "tz_id": "Asia/Karachi", "localtime_epoch": 1755069094, "localtime": "2025-08-13 12:11"}, "current": {"last_updated_epoch": 1755068400, "last_updated": "2025-08-13 12:00", "temp_c": 31.4, "temp_f": 88.6, "is_day": 1, "condition": {"text": "Patchy rain nearby", "icon": "//cdn.weatherapi.com/weather/64x64/day/176.png", "code": 1063}, "wind_mph": 5.4, "wind_kph": 8.6, "wind_degree": 159, "wind_dir": "SSE", "pressure_mb": 1002.0, "pressure_in": 29.59, "precip_mm": 0.06, "precip_in": 0.0, "humidity": 62, "cloud": 75, "feelslike_c": 36.5, "feelslike_f": 97.7, "windchill_c": 31.4, "windchill_f": 88.6, "heatindex_c": 36.5, "heatindex_f": 97.7, "dewpoint_c": 23.3, "dewpoint_f": 74.0, "vis_km": 10.0, "vis_miles": 6.0, "uv": 7.8, "gust_mph": 6.2, "gust_kph": 9.9, "short_rad": 894.64, "diff_rad": 163.95, "dni": 2386.32, "gti": 902.84}}', name='get_weather', tool_call_id

In [ ]:
message.append(ai_message)

In [ ]:
message

[HumanMessage(content='What is the weather in the pakistan in the celcius also convert this celcius into farenheit', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_7v9aAPkIJt0OTbdOEJMTl0U1', 'function': {'arguments': '{"location": "Pakistan"}', 'name': 'get_weather'}, 'type': 'function'}, {'id': 'call_Sne8pxvCBF1CANvFAJuPxi61', 'function': {'arguments': '{}', 'name': 'conver_c_to_f'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 93, 'total_tokens': 137, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C3zwKSPXtHSN2vFw54B3OvOZ70voI', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run

In [ ]:
for tool_call in ai_message.tool_calls:
    if tool_call['name'] == 'get_weather':
        tool_message_1 = get_weather.invoke(tool_call)

        celsius_temp = json.loads(tool_message_1.content)['current']['temp_c']  # Extract Celsius
        message.append(tool_message_1)

    if tool_call['name'] == 'conver_c_to_f':
        tool_call['args']['temperature'] = celsius_temp
        tool_message_2 = conver_c_to_f.invoke(tool_call)
        message.append(tool_message_2)

In [ ]:
message

[HumanMessage(content='What is the weather in the pakistan in the celcius also convert this celcius into farenheit', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_7v9aAPkIJt0OTbdOEJMTl0U1', 'function': {'arguments': '{"location": "Pakistan"}', 'name': 'get_weather'}, 'type': 'function'}, {'id': 'call_Sne8pxvCBF1CANvFAJuPxi61', 'function': {'arguments': '{}', 'name': 'conver_c_to_f'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 93, 'total_tokens': 137, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C3zwKSPXtHSN2vFw54B3OvOZ70voI', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run

In [ ]:
final_result = llm.invoke(message)


In [ ]:
final_result.content

'The weather in Islamabad, Pakistan is currently 31.4°C (88.6°F).'